# 手撕DeepSeek MLA

不考虑效率

仅复现原论文公式

> ref: DeepSeek-V2: A Strong, Economical, and Efficient Mixture-of-Experts Language Model

# setting

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
# @dataclass
class ModelArgs:
    dim: int = 64
    n_heads: int = 8
    n_kv_heads: int =  2

    # down 两者远小于 dim
    dc_kv: int = 4 
    dc_q: int = 4

bs = 3
seq_len = 5

config = ModelArgs()
print(config)

In [3]:
h = torch.randn(bs, seq_len, config.dim)

## Standard Multi-Heads Attention

参考Llama3-GQA， 去除rope简易实现

In [4]:
# ref : Llama3 GQA, ./notebook/Llama3-GQA.ipynb
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """torch.repeat_interleave(x, dim=2, repeats=n_rep)"""
    bs, slen, n_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return (
        x[:, :, :, None, :]
        .expand(bs, slen, n_kv_heads, n_rep, head_dim) # 
        .reshape(bs, slen, n_kv_heads * n_rep, head_dim)
    )

class MultiHeadsAttention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        self.head_dim = args.dim // args.n_heads # 18/6 = 3
        self.n_rep = self.n_heads // self.n_kv_heads

        self.wq = nn.Linear(in_features=args.dim, out_features=args.n_heads * self.head_dim,bias=False,)
        self.wk = nn.Linear(in_features=args.dim, out_features=args.n_kv_heads * self.head_dim,bias=False,)
        self.wv = nn.Linear(in_features=args.dim, out_features=args.n_kv_heads * self.head_dim,bias=False,)
        self.wo = nn.Linear(in_features=args.n_heads * self.head_dim, out_features=args.dim,bias=False,)
        
    def forward(
        self,
        x: torch.Tensor,
    ):
        bsz, seqlen, _ = x.shape
        xq, xk, xv = self.wq(x), self.wk(x), self.wv(x)

        # here we ignore RoPE

        xq = xq.view(bsz, seqlen, self.n_heads, self.head_dim)
        xk = xk.view(bsz, seqlen, self.n_kv_heads, self.head_dim)
        xv = xv.view(bsz, seqlen, self.n_kv_heads, self.head_dim)
        
        keys = repeat_kv( xk, self.n_rep )  
        values = repeat_kv( xv, self.n_rep )  
        
        xq = xq.transpose(1, 2)  # (bs, n_local_heads, seqlen, head_dim)
        keys = keys.transpose(1, 2)  # (bs, n_local_heads, seqlen, head_dim)
        values = values.transpose(1, 2)  # (bs, n_local_heads, seqlen, head_dim)

        
        scores = xq @ keys.transpose(2, 3) / math.sqrt(self.head_dim)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        output = scores @ values # (bs, n_local_heads, seqlen, head_dim)
        output = output.transpose(1, 2).contiguous().view(bsz, seqlen, -1)
        mha_output = self.wo(output)
        return mha_output

In [5]:
mha = MultiHeadsAttention(config)
print(mha)

MultiHeadsAttention(
  (wq): Linear(in_features=64, out_features=64, bias=False)
  (wk): Linear(in_features=64, out_features=16, bias=False)
  (wv): Linear(in_features=64, out_features=16, bias=False)
  (wo): Linear(in_features=64, out_features=64, bias=False)
)


In [6]:
out = mha(h)
print(h.shape)
print(out.shape)

torch.Size([3, 5, 64])
torch.Size([3, 5, 64])


## Multi-Heads Latent Attention

### model

1. 以下用down和up权重矩阵，代替直接的wq,wk,wv
2. MLA矩阵发生于训练之时

In [7]:
class MultiHeadsLatentAttention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.dim = args.dim
        # self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        self.head_dim = args.dim // args.n_heads # 18/6 = 3
        self.dc_kv = args.dc_kv
        self.dc_q = args.dc_q

        # MLA Structure
        self.wq_down = nn.Linear(in_features=args.dim, out_features=args.dc_q, bias=False,)
        self.wq_up = nn.Linear(in_features=args.dc_q, out_features=args.dim , bias=False,)

        self.wkv_down = nn.Linear(in_features=args.dim, out_features=args.dc_kv, bias=False,)
        self.wk_up = nn.Linear(in_features=args.dc_kv, out_features=args.dim, bias=False,)
        self.wv_up = nn.Linear(in_features=args.dc_kv, out_features=args.dim, bias=False,)
        
        self.wo = nn.Linear(in_features=args.dim, out_features=args.dim,bias=False,)


In [8]:
mla = MultiHeadsLatentAttention(config)
print(mla)

MultiHeadsLatentAttention(
  (wq_down): Linear(in_features=64, out_features=4, bias=False)
  (wq_up): Linear(in_features=4, out_features=64, bias=False)
  (wkv_down): Linear(in_features=64, out_features=4, bias=False)
  (wk_up): Linear(in_features=4, out_features=64, bias=False)
  (wv_up): Linear(in_features=4, out_features=64, bias=False)
  (wo): Linear(in_features=64, out_features=64, bias=False)
)


### forward

In [9]:
# 该段代码是关键，先降维，再升维
# xq = mha.wq(h)
c_q = mla.wq_down(h)
xq = mla.wq_up(c_q)

# xk = mha.wk(h)
# xk = mha.wv(h)
c_kv = mla.wkv_down(h)
xk = mla.wk_up(c_kv)
xv = mla.wv_up(c_kv)

print(xq.shape)
print(xk.shape)
print(xv.shape)

torch.Size([3, 5, 64])
torch.Size([3, 5, 64])
torch.Size([3, 5, 64])


In [10]:
# 与传统多头注意力无差别
xq = xq.view(bs, seq_len, mla.n_heads, mla.head_dim)
xk = xk.view(bs, seq_len, mla.n_heads, mla.head_dim)
xv = xv.view(bs, seq_len, mla.n_heads, mla.head_dim)

query = xq.transpose(1,2)
key = xk.transpose(1,2) # bs, n_heads, [seq_len, head_dim]
value = xv.transpose(1,2)
print(query.shape) 
print(key.shape)

scores = query @ key.transpose(2,3) / math.sqrt(mla.head_dim) # keys: bs, n_heads, [head_dim, seq_len]
scores = F.softmax(scores.float(), dim=-1).type_as(xq)
output = scores @ value 
output = output.transpose(1, 2).contiguous().view(bs, seq_len, -1)
output = mla.wo(output)
print(output)

torch.Size([3, 8, 5, 8])
torch.Size([3, 8, 5, 8])
tensor([[[ 4.5146e-02,  1.5168e-01, -1.3621e-02, -1.7782e-02,  2.9059e-02,
          -8.5655e-02,  1.1190e-02, -2.2851e-03, -5.7409e-02,  2.2138e-02,
          -4.4059e-02, -6.3881e-02, -4.2939e-02, -1.9402e-02, -2.1850e-02,
          -9.7752e-02,  1.3288e-01,  1.2167e-02, -4.2248e-02, -4.9612e-02,
           7.4241e-02,  1.2082e-01, -4.5428e-02,  7.9689e-02,  1.0336e-01,
          -4.8504e-02, -8.4458e-02,  1.0638e-02,  6.1907e-02, -1.0487e-01,
          -5.7420e-02,  3.0349e-02, -5.3687e-02,  3.8531e-02,  1.7106e-02,
           9.5486e-02,  2.0908e-03, -1.3231e-01, -7.2697e-02, -1.4581e-01,
          -1.2431e-02, -2.2906e-02, -2.7741e-02,  4.6539e-02,  2.8527e-02,
          -2.7289e-02,  7.3095e-02,  5.9925e-02, -3.6553e-02,  1.1505e-01,
          -1.1351e-01,  2.3730e-02,  8.4835e-02,  7.5701e-02, -8.1039e-02,
           6.9898e-02,  6.0617e-02,  5.8915e-03, -3.7927e-02,  7.3171e-03,
          -3.7854e-02,  9.2171e-03, -8.1242e-02, -

### 矩阵吸收

以上参数发生在训练之时，训练完成后，我们可以做吸收操作，具体指wq参数和Wo参数，

我们写出如下等式：

Q = wq_up * ( wq_down * h ) = (wq_up * wq_down) * h

Q = wq * h 

目的是什么？

1. 训练时省显存
2. 训练完，推理Q满矩阵保精度，
3. 由于KV Cache的存在，decoding阶段时one-by-one token进行q计算
4. KV Cache极具减少存储体现。

In [11]:
wq = mla.wq_up.weight.data @ mla.wq_down.weight.data 

### 矩阵吸收后的forward(非训练阶段）

In [12]:
# 该段代码是关键，先降维，再升维
xq =  h @ wq
# c_q = mla.wq_down(h) #去除
# xq = mla.wq_up(c_q) #去除

c_kv = mla.wkv_down(h)
xk = mla.wk_up(c_kv)
# xv = mla.wv_up(c_kv) # 去除

# 与传统多头注意力无差别
xq = xq.view(bs, seq_len, mla.n_heads, mla.head_dim)
xk = xk.view(bs, seq_len, mla.n_heads, mla.head_dim)
xv = xv.view(bs, seq_len, mla.n_heads, mla.head_dim)

query = xq.transpose(1,2)
key = xk.transpose(1,2) # bs, n_heads, [seq_len, head_dim]
value = xv.transpose(1,2)
print(query.shape) 
print(key.shape)

# keys: bs, n_heads, [head_dim, seq_len]
scores = query @ key.transpose(2,3) / math.sqrt(mla.head_dim) 
scores = F.softmax(scores.float(), dim=-1).type_as(xq)
output = scores @ value 
output = output.transpose(1, 2).contiguous().view(bs, seq_len, -1)
output = mla.wo(output)
print(output)

torch.Size([3, 8, 5, 8])
torch.Size([3, 8, 5, 8])
tensor([[[ 2.0484e-02,  1.5552e-01, -1.5591e-02,  1.4401e-03,  3.6842e-02,
          -8.2618e-02,  2.8318e-02, -2.0483e-02, -1.3989e-02, -8.7396e-03,
          -2.2675e-02, -8.3285e-02, -7.2652e-03, -3.4485e-02, -6.9594e-03,
          -9.4004e-02,  1.0591e-01, -1.8713e-02, -2.8423e-02, -7.4676e-02,
           1.0796e-01,  1.0757e-01, -4.3560e-02,  7.3838e-02,  8.9302e-02,
          -4.2553e-02, -7.6589e-02,  1.4797e-02,  7.5575e-02, -1.0200e-01,
          -5.0200e-02,  1.1170e-02, -4.3665e-02,  3.0674e-02,  1.0298e-02,
           1.0395e-01, -1.8623e-02, -1.2574e-01, -1.0559e-01, -1.4850e-01,
          -4.0997e-02, -2.9282e-02, -8.6471e-03,  4.6638e-02,  1.8475e-02,
          -1.7276e-02,  9.6743e-02,  7.0292e-02, -3.9667e-02,  1.1571e-01,
          -1.1643e-01, -2.2672e-02,  8.6867e-02,  7.5407e-02, -1.0060e-01,
           5.2397e-02,  4.3059e-02, -1.0472e-02, -4.3900e-02,  1.2820e-02,
          -5.1049e-02,  5.7975e-03, -8.0848e-02, -

In [13]:
import torch

# 假设 tensor 是形状为 (a, b, c) 的张量
tensor = torch.randn(2, 3, 4)

# 扩展第 1 维，复制 8 份
expanded_tensor = tensor.repeat(8, 1, 1)
print(expanded_tensor.shape)

torch.Size([16, 3, 4])


### KV cache存的是什么？

传统MHA的KV Cache大小: `[2, bs, seq_len, n_kv_head * head_dim]`, 其中2代表K和V

如果我们存储MLA up后的矩阵K,V cache，那么与MHA存储无差别

那么我们可以存储kv down的矩阵：即 c_kv = w_kv_down @ h, 此时：

`[1, bs, seq_len, dc_kv]`

如果原来  2 * n_kv_head * head_dim = dim = 2 * 4096, 如果dc_kv  = 512

那么MLA KV-cache就为MHA的 1 / 16

### MLA压缩的本质是什么？

计算时间换空间

存储时刻，w_k_up

decoding时刻 即将：

`K = w_k_up @ c_kv`

`V = w_v_up @ c_kv`

压缩KV-Cache的量的意义？以VLLM来说，减少KV-Cache量。推理服务能跑更大的batch-size，从而提高inference，decoding的效率

### MLA下位置编码问题

1. 常规算法为：RoPE(W_k_up( w_k_down(h) )), 这里的RoPE没法低秩分解
2. 可以写为attention： `Rk @ Wkup @ wkdown(h)^T` `@` `[Rq @ Wqup @ wqdown(h)]^T` 
3. 上式注意到，我们所存储kv cache是wkdown(h)为 `seq x dc_kv`, 那么我们每次计算出了要up，而且还要对**所有token**增加RoPE操作
4. 有什么方式可以减少RoPE操作吗？解决方案为在h上，增加额外的参数矩阵比如：

In [14]:
class MultiHeadsLatentAttention_withRoPE(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.dim = args.dim
        # self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        self.head_dim = args.dim // args.n_heads # 18/6 = 3
        self.dc_kv = args.dc_kv
        self.dc_q = args.dc_q

        # MLA Structure
        self.wq_down = nn.Linear(in_features=self.dim, out_features=self.dc_q, bias=False,)
        self.wq_up = nn.Linear(in_features=self.dc_q, out_features=self.dim , bias=False,)

        self.wkv_down = nn.Linear(in_features=self.dim, out_features=self.dc_kv, bias=False,)
        self.wk_up = nn.Linear(in_features=self.dc_kv, out_features=self.dim, bias=False,)
        self.wv_up = nn.Linear(in_features=self.dc_kv, out_features=self.dim, bias=False,)
        
        self.wo = nn.Linear(in_features=self.dim, out_features=self.dim,bias=False,)

        # RoPE Weight
        # K 每头一样， Q每头不一样
        self.wq_up_rope = nn.Linear(in_features=self.dc_q, out_features=self.dim , bias=False,)
        self.wk_head_rope = nn.Linear(in_features=self.dim, out_features=self.head_dim , bias=False,)

In [15]:
mla_rope = MultiHeadsLatentAttention_withRoPE(config)
print(mla_rope)

MultiHeadsLatentAttention_withRoPE(
  (wq_down): Linear(in_features=64, out_features=4, bias=False)
  (wq_up): Linear(in_features=4, out_features=64, bias=False)
  (wkv_down): Linear(in_features=64, out_features=4, bias=False)
  (wk_up): Linear(in_features=4, out_features=64, bias=False)
  (wv_up): Linear(in_features=4, out_features=64, bias=False)
  (wo): Linear(in_features=64, out_features=64, bias=False)
  (wq_up_rope): Linear(in_features=4, out_features=64, bias=False)
  (wk_head_rope): Linear(in_features=64, out_features=8, bias=False)
)


In [16]:
# 增加rope
c_q = mla_rope.wq_down(h)
xq = mla_rope.wq_up(c_q)

c_kv = mla_rope.wkv_down(h)
xk = mla_rope.wk_up(c_kv)
xv = mla_rope.wv_up(c_kv)

# print(xq.shape)
# print(xk.shape)
# print(xv.shape)

# 位置编码相关
r_q = mla_rope.wq_up_rope(c_q) #多头
r_k = mla_rope.wk_head_rope(h) #单头
print(r_q.shape)
print(r_k.shape)

torch.Size([3, 5, 64])
torch.Size([3, 5, 8])


产生新的疑问，r_q 和 r_k 维度不同，那么做rope的dim如何处理？

In [17]:
# 简易写下
rope_matrix_q = [torch.randn(mla_rope.dim, mla_rope.dim)] * seq_len
rope_matrix_k = [torch.randn(mla_rope.head_dim, mla_rope.head_dim)] * seq_len
def apply_rope_q(x, seq_len, rope_matrix):
    for i in range(seq_len):
        x[:, i, :] = x[:, i, :] @ rope_matrix[i]
    return x

rope_q = apply_rope_q(r_q, seq_len, rope_matrix_q)
rope_k = apply_rope_q(r_k, seq_len, rope_matrix_k)

对于q每头，cat不一样的位置信息

对于k每头，cat一样的信息

In [18]:
# 与传统多头注意力无差别
xq = xq.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim)
xk = xk.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim)
xv = xv.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim)


query = xq.transpose(1,2)
key = xk.transpose(1,2) # bs, n_heads, [seq_len, head_dim]
value = xv.transpose(1,2)

print(query.shape) 
print(key.shape)

## 嵌入rope
rope_q_head = rope_q.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim)
rope_q_head = rope_q_head.transpose(1,2)

rope_k_head = rope_k.unsqueeze(dim = 1).repeat( repeats = [1, mla_rope.n_heads, 1, 1])
print(rope_q_head.shape)
print(rope_k_head.shape)


# cat 操作
query_cat = torch.cat((query, rope_q_head), dim = -1)
key_cat = torch.cat((query, rope_k_head), dim = -1)

torch.Size([3, 8, 5, 8])
torch.Size([3, 8, 5, 8])
torch.Size([3, 8, 5, 8])
torch.Size([3, 8, 5, 8])


In [19]:
### 常规attention
# scores = query @ key.transpose(2,3) # keys: bs, n_heads, [head_dim, seq_len]
# keys: bs, n_heads, [2head_dim, seq_len]
scores = query_cat @ key_cat.transpose(2,3) / math.sqrt(2 * mla_rope.head_dim) # cat后dim维度变化，底数也有变化 
scores = F.softmax(scores.float(), dim=-1).type_as(xq)
output = scores @ value 
output = output.transpose(1, 2).contiguous().view(bs, seq_len, -1)
output = mla.wo(output)
print(output.shape)

torch.Size([3, 5, 64])


## 增加问题

1. 为什么位置编码要分离
2. q和k的位置编码维度不一样，那么rope的维度是否一样
3. v_up 如何被 wo 吸收
5. 写出带kv_cache版本的MLA，及decoding代码
6. MLA训练的显存计算
7. MLA并行参数分配、张量并行细节和通信

### 答案1

1. 位置编码敏感
2. 不分离的话，inference阶段要对kv做重复rope变换，
3. 如果分离那么直接拼接算好的rope k(我理解其实位置编码rope k也应该是cache起来的）

原论文

> To be specific, RoPE is position-sensitive for both keys and queries

> As a result, we must recompute the keys for all the prefix tokens during inference, which will significantly hinder the inference efficiency

> During inference, the decoupled key should also be cached.

### 答案2:

>  Therefore, DeepSeek-V2 requires a total KV cache containing $(𝑑_𝑐 + 𝑑^𝑅_ℎ)𝑙 $ elements.
1. 先说KV cache量是多少, l是长度，d_c是latent维度，$d^R_h$是单头的向量维度

2. 为什么QK位置编码有区别，Q是多头$d^R_h * h$, h是头数，那么多头保精度

3. rope K是单头，从cache角度来看，存单头k rope是经济的，随着上下文长度增加，如果是k rope多头维度本质上与常规KV无差别


### 答案3

原论文公式

$$
\begin{align}
   [\mathbf{v}_{t, 1}^{C};\mathbf{v}_{t, 2}^{C};...;\mathbf{v}_{t, n_{h}}^{C}] = \mathbf{v}_{t}^{C} &= W^{UV} \mathbf{c}_{t}^{KV}, \\
    \mathbf{o}_{t, i} &= \sum_{j=1}^{t} \operatorname{Softmax}_j(\frac{\mathbf{q}_{t, i}^T \mathbf{k}_{j, i}}{\sqrt{d_{h} + d_{h}^{R}}}) \mathbf{v}_{j, i}^{C}, \\
    \mathbf{u}_{t} &= W^{O} [\mathbf{o}_{t, 1};\mathbf{o}_{t, 2};...;\mathbf{o}_{t, n_{h}}],
\end{align}
$$

关于吸收, 原论文提到uv吸收到wo

> Fortunately, due to the associative law of matrix multiplication, we can absorb $W^{UK}$ into $W^{UQ}$, and $W^{UV}$ into $W^{O}$. 

In [23]:
# 编程举例 WUV 到 WO的矩阵吸收

c = 64
d = 4096
seq_len = 16 # length

c_kv = torch.randn(seq_len, c)
W_O = torch.randn(d, d)
W_UV = torch.randn(c, d)

Q = torch.randn(seq_len, d) # ignore rope q
K = torch.randn(seq_len, d) # ignore rope k

# 原论文公式代码，忽略底数 
V = c_kv @ W_UV
S = Q @ K.t()  
P = F.softmax(S, dim = -1)
O = P @ V
U = O @ W_O
print(U[:5,:5])

# 吸收版本
w_uv_absord = W_UV @ W_O  
# 那么实际部署时，就保留这个参数矩阵 w_uv_absord。
# 原本存 dxd + cxd, 那么现在只存 cxd (减少的参数量体感非常明显)

V = c_kv @ w_uv_absord
S = Q @ K.t()  
P = F.softmax(S, dim = -1)
O = P @ V
# U = O @ W_o # 不需要再变换了
print(O[:5,:5])

tensor([[ 586.4882,  -14.7992, -678.4225,  116.2531, -610.6373],
        [ -59.8604, -180.5592, -280.9830, -789.0940, -244.8712],
        [-199.1532,  982.1359, -158.7461,  496.4213,   89.5527],
        [ -59.8604, -180.5592, -280.9830, -789.0940, -244.8712],
        [-199.1532,  982.1359, -158.7462,  496.4213,   89.5527]])
tensor([[ 586.4888,  -14.8000, -678.4230,  116.2542, -610.6379],
        [ -59.8597, -180.5598, -280.9839, -789.0929, -244.8703],
        [-199.1532,  982.1357, -158.7457,  496.4212,   89.5529],
        [ -59.8597, -180.5598, -280.9839, -789.0929, -244.8703],
        [-199.1532,  982.1357, -158.7457,  496.4212,   89.5529]])


### 答案4

1. 分析有哪些矩阵可以吸收

> Fortunately, due to the associative law of matrix multiplication, we can absorb $W^{UK}$ into $W^{UQ}$, and $W^{UV}$ into $W^{O}$.

2. 另外q的矩阵可以吸收，q矩阵除了常规的 wq x = Q 之外， 还得做一次 wq_rope x = q_rope, 增加一次矩阵乘

3. kv cache要存 1. `rope k`， 2.latent `C`


### 答案5复杂，先skip

### 答案6分析

#### 切分方案A

1. 先说down 参数矩阵， Wk_down ( d x c ), 如果按照c切分，每个GPU 存储 wk_down_shard (d x c'), 输出 h(l,d) -> C(l, c')

2. up 矩阵，wk_up (c x d), 按照c切分, 每个gpu存储 wk_up_shard(c' x d) ，每个GPU输出   C(l, c') x  (c', d) -> K(l, d)

这里就有问题K在每个GPU上都是满头的， 重新切分：

#### 切分方案B 

1. 先说down 参数矩阵， Wk_down ( d x c ), 如果按照c切分，每个GPU 存储 wk_down_shard (d x c'), 输出 h(l,d) -> C(l, c')

2. 将每个 C(l, c') 分发给其他GPU， 那么每块GPU都有完整的 C(l, c)， # 考虑到c‘是非常小量的, 比如c是64， c‘是8

3. up 矩阵，wk_up (c x d), 按照d切分,  每个gpu存储 wk_up_shard(c x d'), 每个GPU输出 C(l, c) x  (c, d') -> K(l, d')

所以这样，每个头独立存在各GPU上，那么就可以head-parallel算 多头注意力了。

- 待分析， 上述未分析是否backward通信友好


依次类推可以设计出MLA的tensor parallel架构
